## Machine Learning Classifaction
1. Support Vector Machine SVM

In [15]:
from sklearn import svm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Since the advatnage of SVM is that it is 
#very effective in high dimensional spaces,
#We will not use PCA on the training sets 
#to reduce the number of dimensions for SVM

#From all the PhotoObj parameters, for the first run we'll be using all 
#parameters that are not RA, DEC and Errors.
#Due to that we have edited the PhotoObj parameter to only include 
#the parameters that are usefull for our classification attempt with SVM
    
#Opening the selected param text file 
svmParam = open('./machine-learning-sets/dan.txt','r')
    
svmList = svmParam.readlines()
svmParam.close()

#Convert the selected param text file into a more readible format for
#crossID which wants no new lines containing only a ',' to sepereate
#parameters
svmParam_no_nl = open("./machine-learning-sets/SVM_params_no_nl.txt","w+")

for n in svmList:
    svmParam_no_nl.write(n.replace('\n',''))

svmParam_no_nl.readlines()
    
svmParam_no_nl.close()

In [16]:
#Now we go back to SDSS crossid with this paramList & the two 
#foreground and background data set and retrieve all the
#parameters

#First need to extract only name, RA, DEC from the two data sets
#for SDSS CrossID
base_foreground = pd.read_csv('./complete/dwarfs.csv')
base_foreground.head()

,objID,ra,dec,run,rerun,camcol,field,type,u0,g0,r0,i0,z0,Redshift
0,1237680311772250464,16.765005,32.389555,8111,301,5,174,GALAXY,17.14250,16.04856,15.63828,15.42416,15.29288,0.016998
1,1237680315521237319,16.792826,32.346872,8112,301,4,35,GALAXY,18.42509,16.79932,16.11763,15.77143,15.49772,0.014160
2,1237680311772250198,16.685466,32.418586,8111,301,5,174,GALAXY,18.37703,16.79943,16.09405,15.74896,15.51344,0.020514
3,1237680315521302864,16.966691,32.256108,8112,301,4,36,GALAXY,17.90406,16.35933,15.61268,15.26653,15.02099,0.018193
4,1237680311772381486,17.066149,32.499011,8111,301,5,176,GALAXY,18.33514,16.66942,15.94401,15.56867,15.34149,0.018373


In [17]:
cross_id_foreground = base_foreground[['objID','ra','dec']]
cross_id_foreground.head()

,objID,ra,dec
0,1237680311772250464,16.765005,32.389555
1,1237680315521237319,16.792826,32.346872
2,1237680311772250198,16.685466,32.418586
3,1237680315521302864,16.966691,32.256108
4,1237680311772381486,17.066149,32.499011


In [18]:
base_background = pd.read_csv('./complete/background.csv')
base_background.head()

,ObjectName,objID,ra,dec,run,rerun,camcol,field,type,u0,g0,r0,i0,z0,Redshift,RedshiftFlag
0,2MASXJ01070798+3223335,1237680311772250504,16.783124,32.392580,8111,301,5,174,GALAXY,20.13362,18.32410,17.19614,16.74197,16.40898,0.153931,SPEC
1,2MASXJ01065749+3213214,1237680315521237275,16.739504,32.222824,8112,301,4,35,GALAXY,18.94812,17.26759,16.39990,15.94577,15.60334,0.114617,SPEC
2,IVZw038NOTES01,1237680311772250292,16.624039,32.507760,8111,301,5,174,GALAXY,20.15441,18.22609,17.11155,16.65807,16.33635,0.153832,SPEC
3,2MASXJ01082662+3229109,1237680311772381566,17.110951,32.486381,8111,301,5,176,GALAXY,19.56098,17.58059,16.57519,16.14223,15.82258,0.111395,SPEC
4,IVZw038NOTES03,1237680316058107970,16.737807,32.622441,8112,301,5,35,GALAXY,18.35271,16.36183,15.47458,15.03607,14.70395,0.066042,NaN


In [19]:
cross_id_background = base_background[['objID','ra','dec']]
cross_id_background.head()

,objID,ra,dec
0,1237680311772250504,16.783124,32.392580
1,1237680315521237275,16.739504,32.222824
2,1237680311772250292,16.624039,32.507760
3,1237680311772381566,17.110951,32.486381
4,1237680316058107970,16.737807,32.622441


In [20]:
#Exporting the two files to use in CROSSID
cross_id_foreground.to_csv(r'./machine-learning-sets/foreground_crossID.csv', index=False)
cross_id_background.to_csv(r'./machine-learning-sets/background_crossID.csv', index=False)

SQL used to grab all the PhotoObj parameters that are relevant based on the SVM_params_no_nl.txt file. This SQL works but be patient, it takes a while to grab all the parameter for each element. All the magnitudes in this SQL search have ben de-reded by subtracting the respective extinction value from it.

```SQL
SELECT 
(p.psfMag_u - p.extinction_u) as psfMag_u0,(p.psfMag_g - p.extinction_g) as psfMag_g0,(p.psfMag_r - p.extinction_r) as psfMag_r0,(p.psfMag_i - p.extinction_i) as psfMag_i0,(p.psfMag_z - p.extinction_z) as psfMag_z0,(p.fiberMag_u - p.extinction_u) as fiberMag_u0,(p.fiberMag_g - p.extinction_g) as fiberMag_g0,(p.fiberMag_r - p.extinction_r) as fiberMag_r0,(p.fiberMag_i - p.extinction_i) as fiberMag_i0,(p.fiberMag_z - p.extinction_z) as fiberMag_z0,(p.fiber2Mag_u - p.extinction_u) as fiber2Mag_u0,(p.fiber2Mag_g - p.extinction_g) as fiber2Mag_g0,(p.fiber2Mag_r - p.extinction_r) as fiber2Mag_r0,(p.fiber2Mag_i - p.extinction_i) as fiber2Mag_i0,(p.fiber2Mag_z - p.extinction_z) as fiber2Mag_z0,(p.petroMag_u - p.extinction_u) as petroMag_u0,(p.petroMag_g - p.extinction_g) as petroMag_g0,(p.petroMag_r - p.extinction_r) as petroMag_r0,(p.petroMag_i - p.extinction_i) as petroMag_i0,(p.petroMag_z - p.extinction_z) as petroMag_z0,p.petroRad_u,p.petroRad_g,p.petroRad_r,p.petroRad_i,p.petroRad_z,p.petroR50_u,p.petroR50_g,p.petroR50_r,p.petroR50_i,p.petroR50_z,p.petroR90_u,p.petroR90_g,p.petroR90_r,p.petroR90_i,p.petroR90_z,p.q_u,p.q_g,p.q_r,p.q_i,p.q_z,p.u_u,p.u_g,p.u_r,p.u_i,p.u_z,p.mE1_u,p.mE1_g,p.mE1_r,p.mE1_i,p.mE1_z,p.mE2_u,p.mE2_g,p.mE2_r,p.mE2_i,p.mE2_z,p.mRrCc_u,p.mRrCc_g,p.mRrCc_r,p.mRrCc_i,p.mRrCc_z,p.mCr4_u,p.mCr4_g,p.mCr4_r,p.mCr4_i,p.mCr4_z,p.mE1PSF_u,p.mE1PSF_g,p.mE1PSF_r,p.mE1PSF_i,p.mE1PSF_z,p.mE2PSF_u,p.mE2PSF_g,p.mE2PSF_r,p.mE2PSF_i,p.mE2PSF_z,p.mRrCcPSF_u,p.mRrCcPSF_g,p.mRrCcPSF_r,p.mRrCcPSF_i,p.mRrCcPSF_z,p.mCr4PSF_u,p.mCr4PSF_g,p.mCr4PSF_r,p.mCr4PSF_i,p.mCr4PSF_z,p.deVRad_u,p.deVRad_g,p.deVRad_r,p.deVRad_i,p.deVRad_z,p.deVAB_u,p.deVAB_g,p.deVAB_r,p.deVAB_i,p.deVAB_z,(p.deVMag_u - p.extinction_u) as deVMag_u0,(p.deVMag_g - p.extinction_g) as deVMag_g0,(p.deVMag_r - p.extinction_r) as deVMag_r0,(p.deVMag_i - p.extinction_i) as deVMag_i0,(p.deVMag_z - p.extinction_z) as deVMag_z0,p.expRad_u,p.expRad_g,p.expRad_r,p.expRad_i,p.expRad_z,p.expAB_u,p.expAB_g,p.expAB_r,p.expAB_i,p.expAB_z,(p.expMag_u - p.extinction_u) as expMag_u0,(p.expMag_g - p.extinction_g) as expMag_g0,(p.expMag_r - p.extinction_r) as expMag_r0,(p.expMag_i - p.extinction_i) as expMag_i0,(p.expMag_z - p.extinction_z) as expMag_z0,(p.cModelMag_u - p.extinction_u) as cModelMag_u0,(p.cModelMag_g - p.extinction_g) as cModelMag_g0,(p.cModelMag_r - p.extinction_r) as cModelMag_r0,(p.cModelMag_i - p.extinction_i) as cModelMag_i0,(p.cModelMag_z - p.extinction_z) as cModelMag_z0,p.aperFlux7_u,p.aperFlux7_g,p.aperFlux7_r,p.aperFlux7_i,p.aperFlux7_z,p.fracDeV_u,p.fracDeV_g,p.fracDeV_r,p.fracDeV_i,p.fracDeV_z,p.ra,p.dec,p.dered_u,p.dered_g,p.dered_r,p.dered_i,p.dered_z
FROM #upload u
      JOIN #x x ON x.up_id = u.up_id
      JOIN PhotoObj p ON p.objID = x.objID 
ORDER BY x.up_id
```


In [21]:
#The Background Dataset
background = pd.read_csv('./machine-learning-sets/background_FULL_SDSS.csv')

#Droping the unuseful objID that comes with CrossID
background = background.drop(['objID','objID1'], axis=1)

#Adding the column that labels these as the background
#In this case the label for background is 0 and 
#foreground will be 1
background['label'] = 0

background.head()

,psfMag_u0,psfMag_g0,psfMag_r0,psfMag_i0,psfMag_z0,fiberMag_u0,fiberMag_g0,fiberMag_r0,fiberMag_i0,fiberMag_z0,...,fracDeV_i,fracDeV_z,ra,dec,dered_u,dered_g,dered_r,dered_i,dered_z,label
0,21.08928,19.37086,18.25864,17.81117,17.51476,21.25800,19.36015,18.24822,17.80149,17.45830,...,1.000000,1.000000,16.783124,32.392580,20.13362,18.32410,17.19614,16.74197,16.40898,0
1,21.41094,19.66853,19.04671,18.49730,18.12082,21.32957,19.64395,18.69971,18.21872,17.81517,...,0.102860,0.200284,16.739504,32.222824,18.94811,17.26758,16.39990,15.94577,15.60334,0
2,21.27979,19.46498,18.33810,17.92343,17.63418,21.25036,19.44884,18.34415,17.88955,17.55409,...,1.000000,1.000000,16.624039,32.507760,20.15441,18.22609,17.11155,16.65807,16.33635,0
3,20.79575,18.89698,17.82768,17.38629,17.21747,20.79277,18.79881,17.79642,17.37555,17.02053,...,0.997685,1.000000,17.110951,32.486381,19.56097,17.58059,16.57519,16.14223,15.82258,0
4,19.69155,17.76127,16.98154,16.55225,16.24577,19.77281,17.82529,16.89516,16.47335,16.13554,...,0.954829,0.975430,16.737807,32.622441,18.35271,16.36183,15.47458,15.03607,14.70395,0


In [22]:
#The Foreground Dataset
foreground = pd.read_csv('./machine-learning-sets/foreground_FULL_SDSS.csv')

#Droping the unuseful objID that comes with CrossID
foreground = foreground.drop(['objID','objID1'], axis=1)

#Adding the column that labels these as the background
#In this case the label for background is 0 and 
#foreground will be 1
foreground['label'] = 1

foreground.head()

,psfMag_u0,psfMag_g0,psfMag_r0,psfMag_i0,psfMag_z0,fiberMag_u0,fiberMag_g0,fiberMag_r0,fiberMag_i0,fiberMag_z0,...,fracDeV_i,fracDeV_z,ra,dec,dered_u,dered_g,dered_r,dered_i,dered_z,label
0,20.68459,20.32490,19.90491,19.65816,19.58863,20.93056,19.84648,19.39435,19.13173,18.93974,...,0.000000,0.024766,16.765005,32.389555,17.14250,16.04856,15.63828,15.42416,15.29288,1
1,20.38590,18.79545,18.40734,18.01583,17.85035,20.31682,18.76537,18.05824,17.72758,17.45851,...,0.262290,0.388589,16.792826,32.346872,18.42509,16.79932,16.11763,15.77143,15.49772,1
2,20.37486,19.15429,18.46381,18.16368,18.04567,20.36893,18.84602,18.14171,17.79829,17.57360,...,0.215467,0.192362,16.685466,32.418586,18.37703,16.79943,16.09405,15.74896,15.51344,1
3,20.10566,18.50956,17.90357,17.60673,17.37036,20.09557,18.56411,17.81530,17.47781,17.24816,...,0.876894,0.829082,16.966691,32.256108,17.90406,16.35933,15.61268,15.26653,15.02099,1
4,20.55464,18.93523,18.13289,17.79406,17.84563,20.42518,18.61701,17.89809,17.54489,17.28442,...,0.392165,0.397820,17.066149,32.499011,18.33514,16.66942,15.94401,15.56867,15.34149,1


In [23]:
#The Foreground and Backgroun Sets now have to be concated together and have
#the label column removed as a seperate array to perform SVM
training_combine = pd.concat([foreground,background])
training_combine.head()

,psfMag_u0,psfMag_g0,psfMag_r0,psfMag_i0,psfMag_z0,fiberMag_u0,fiberMag_g0,fiberMag_r0,fiberMag_i0,fiberMag_z0,...,fracDeV_i,fracDeV_z,ra,dec,dered_u,dered_g,dered_r,dered_i,dered_z,label
0,20.68459,20.32490,19.90491,19.65816,19.58863,20.93056,19.84648,19.39435,19.13173,18.93974,...,0.000000,0.024766,16.765005,32.389555,17.14250,16.04856,15.63828,15.42416,15.29288,1
1,20.38590,18.79545,18.40734,18.01583,17.85035,20.31682,18.76537,18.05824,17.72758,17.45851,...,0.262290,0.388589,16.792826,32.346872,18.42509,16.79932,16.11763,15.77143,15.49772,1
2,20.37486,19.15429,18.46381,18.16368,18.04567,20.36893,18.84602,18.14171,17.79829,17.57360,...,0.215467,0.192362,16.685466,32.418586,18.37703,16.79943,16.09405,15.74896,15.51344,1
3,20.10566,18.50956,17.90357,17.60673,17.37036,20.09557,18.56411,17.81530,17.47781,17.24816,...,0.876894,0.829082,16.966691,32.256108,17.90406,16.35933,15.61268,15.26653,15.02099,1
4,20.55464,18.93523,18.13289,17.79406,17.84563,20.42518,18.61701,17.89809,17.54489,17.28442,...,0.392165,0.397820,17.066149,32.499011,18.33514,16.66942,15.94401,15.56867,15.34149,1


In [24]:
#The two seperate array used in SVM to construct a 
#predicting model
samples = training_combine.drop('label',1)
labels = training_combine['label']

#Using the default kernel type which is a Gaussian
#The other 2 kernel types that might be helpful in this classifcation
#is sigmoid and polynomial
model = svm.SVC(gamma='scale')

#Fitting the model and it can now be used to predict labels if we feed an array with
#the same parameters
model.fit(samples,labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [25]:
#We now grab the test data which is the all-sky csv file retrieved from NED
base_test = pd.read_csv('./complete/all-sky-dwarfs.csv')

#Strip all the unusefull data and format it to be usefull in SDSS CrossID
cross_id_test = base_test[['objID','ra','dec']]

#Export formated file to be used in SDSS CrossID
cross_id_test.to_csv(r'./machine-learning-sets/testData_crossID.csv', index=False)

#After doing CrossID, importing the Testing Dataset
test_Data = pd.read_csv('./machine-learning-sets/testData_FULL_SDSS.csv')

#Droping the unuseful objID that comes with CrossID
test_Data = test_Data.drop(['objID','objID1'], axis=1)

In [26]:
#Time for the predict, remember that 1 means foreground and 0 means background
results = model.predict(test_Data)
print('Number of foregrounds: {}'.format(np.count_nonzero(results == 1)))
print('Number of backgrounds: {}'.format(np.count_nonzero(results == 0)))

Number of foregrounds: 345
Number of backgrounds: 2


In [27]:
#Iteration of the model using the sigmoid kernel rather than the rbf gaussian one
sigmoidModel = svm.SVC(gamma='scale', kernel='sigmoid')
sigmoidModel.fit(samples,labels)
sigResults = sigmoidModel.predict(test_Data)
print('Number of foregrounds with sigmoid kernel: {}'.format(np.count_nonzero(sigResults == 1)))
print('Number of backgrounds with sigmoid kernel: {}'.format(np.count_nonzero(sigResults == 0)))

Number of foregrounds with sigmoid kernel: 345
Number of backgrounds with sigmoid kernel: 2


In [28]:
#Iteration of the model using a polynomial kernel of degree, d
d = 20
polyModel = svm.SVC(gamma='scale', kernel='poly', degree=d)
polyModel.fit(samples,labels)
polyResults = polyModel.predict(test_Data)
print('Number of foregrounds with poly kernel at degree {}: {}'.format(d,np.count_nonzero(polyResults == 1)))
print('Number of backgrounds with poly kernel degree {}: {}'.format(d,np.count_nonzero(polyResults == 0)))

Number of foregrounds with poly kernel at degree 20: 346
Number of backgrounds with poly kernel degree 20: 1
